# 导入所需库
导入必要的Python库，如pandas、numpy、scikit-learn等。

In [ ]:
# 导入必要的Python库
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 加载和预处理历史数据
加载历史电力需求和价格数据，并进行数据清洗和预处理。

In [ ]:
# 加载历史电力需求和价格数据
data = pd.read_csv('historical_power_demand_price.csv')

# 查看数据的前几行
print(data.head())

# 检查数据的基本信息
print(data.info())

# 检查数据是否有缺失值
print(data.isnull().sum())

# 填充或删除缺失值
data = data.fillna(method='ffill')

# 将日期列转换为日期时间格式
data['date'] = pd.to_datetime(data['date'])

# 设置日期列为索引
data.set_index('date', inplace=True)

# 查看数据的统计信息
print(data.describe())

# 数据标准化
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# 将标准化后的数据转换为DataFrame
scaled_data = pd.DataFrame(scaled_data, columns=data.columns, index=data.index)

# 查看标准化后的数据
print(scaled_data.head())

# 特征工程
提取影响电力需求和价格的关键因子，如季节变化、天气状况和市场政策变动。

In [ ]:
# 特征工程

# 提取季节特征
scaled_data['month'] = scaled_data.index.month
scaled_data['day'] = scaled_data.index.day
scaled_data['day_of_week'] = scaled_data.index.dayofweek

# 提取天气特征（假设数据中包含温度和湿度）
scaled_data['temperature'] = data['temperature']
scaled_data['humidity'] = data['humidity']

# 提取市场政策变动特征（假设数据中包含政策变动的标记）
scaled_data['policy_change'] = data['policy_change']

# 查看添加特征后的数据
print(scaled_data.head())

# 构建预测模型
使用机器学习算法（如线性回归、决策树、随机森林等）构建电力需求和价格预测模型。

In [ ]:
# 构建预测模型

# 分割数据集为训练集和测试集
X = scaled_data.drop(['power_demand', 'price'], axis=1)
y_demand = scaled_data['power_demand']
y_price = scaled_data['price']

X_train, X_test, y_demand_train, y_demand_test, y_price_train, y_price_test = train_test_split(
    X, y_demand, y_price, test_size=0.2, random_state=42
)

# 训练电力需求预测模型（线性回归）
demand_model = LinearRegression()
demand_model.fit(X_train, y_demand_train)

# 预测电力需求
y_demand_pred = demand_model.predict(X_test)

# 评估电力需求预测模型
demand_mse = mean_squared_error(y_demand_test, y_demand_pred)
demand_r2 = r2_score(y_demand_test, y_demand_pred)

print(f'电力需求预测模型的均方误差: {demand_mse}')
print(f'电力需求预测模型的R^2值: {demand_r2}')

# 训练电力价格预测模型（随机森林）
from sklearn.ensemble import RandomForestRegressor

price_model = RandomForestRegressor(n_estimators=100, random_state=42)
price_model.fit(X_train, y_price_train)

# 预测电力价格
y_price_pred = price_model.predict(X_test)

# 评估电力价格预测模型
price_mse = mean_squared_error(y_price_test, y_price_pred)
price_r2 = r2_score(y_price_test, y_price_pred)

print(f'电力价格预测模型的均方误差: {price_mse}')
print(f'电力价格预测模型的R^2值: {price_r2}')

# 模型训练与验证
在历史数据的基础上训练模型，并使用验证集评估模型的预测性能。

In [ ]:
# 模型训练与验证

# 分割数据集为训练集和验证集
X_train, X_val, y_demand_train, y_demand_val, y_price_train, y_price_val = train_test_split(
    X_train, y_demand_train, y_price_train, test_size=0.2, random_state=42
)

# 训练电力需求预测模型（线性回归）
demand_model = LinearRegression()
demand_model.fit(X_train, y_demand_train)

# 验证电力需求预测模型
y_demand_val_pred = demand_model.predict(X_val)

# 评估电力需求预测模型
demand_val_mse = mean_squared_error(y_demand_val, y_demand_val_pred)
demand_val_r2 = r2_score(y_demand_val, y_demand_val_pred)

print(f'电力需求预测模型的验证均方误差: {demand_val_mse}')
print(f'电力需求预测模型的验证R^2值: {demand_val_r2}')

# 训练电力价格预测模型（随机森林）
price_model = RandomForestRegressor(n_estimators=100, random_state=42)
price_model.fit(X_train, y_price_train)

# 验证电力价格预测模型
y_price_val_pred = price_model.predict(X_val)

# 评估电力价格预测模型
price_val_mse = mean_squared_error(y_price_val, y_price_val_pred)
price_val_r2 = r2_score(y_price_val, y_price_val_pred)

print(f'电力价格预测模型的验证均方误差: {price_val_mse}')
print(f'电力价格预测模型的验证R^2值: {price_val_r2}')

# 多目标优化
整合经济性、环保目标、用户满意度及价格敏感性等因素，采用多目标优化技术找到最优平衡。

In [ ]:
# 导入必要的库
from scipy.optimize import minimize

# 定义目标函数
def objective_function(x, demand_model, price_model, carbon_cost, user_satisfaction_weight):
    # x[0]: 电力生成成本
    # x[1]: 碳排放成本
    # x[2]: 用户满意度
    # x[3]: 价格敏感性
    
    # 预测电力需求和价格
    predicted_demand = demand_model.predict([x])[0]
    predicted_price = price_model.predict([x])[0]
    
    # 计算总成本
    total_cost = x[0] + carbon_cost * x[1]
    
    # 计算用户满意度
    user_satisfaction = user_satisfaction_weight * x[2]
    
    # 计算价格敏感性
    price_sensitivity = x[3] * predicted_price
    
    # 目标函数：总成本 - 用户满意度 + 价格敏感性
    return total_cost - user_satisfaction + price_sensitivity

# 定义约束条件
def constraint1(x):
    # 电力生成成本必须大于0
    return x[0]

def constraint2(x):
    # 碳排放成本必须大于0
    return x[1]

def constraint3(x):
    # 用户满意度必须大于0
    return x[2]

def constraint4(x):
    # 价格敏感性必须大于0
    return x[3]

# 初始猜测值
x0 = [1, 1, 1, 1]

# 约束条件
constraints = [{'type': 'ineq', 'fun': constraint1},
               {'type': 'ineq', 'fun': constraint2},
               {'type': 'ineq', 'fun': constraint3},
               {'type': 'ineq', 'fun': constraint4}]

# 优化
solution = minimize(objective_function, x0, args=(demand_model, price_model, 0.1, 0.5), constraints=constraints)

# 输出优化结果
print('优化结果:')
print('电力生成成本:', solution.x[0])
print('碳排放成本:', solution.x[1])
print('用户满意度:', solution.x[2])
print('价格敏感性:', solution.x[3])

# 预测与分析
使用训练好的模型进行电力需求和价格的预测，并分析预测结果。

In [ ]:
# 预测与分析

# 使用训练好的电力需求预测模型进行预测
y_demand_pred = demand_model.predict(X_test)

# 使用训练好的电力价格预测模型进行预测
y_price_pred = price_model.predict(X_test)

# 将预测结果与实际值进行比较
results = pd.DataFrame({
    'Actual Demand': y_demand_test,
    'Predicted Demand': y_demand_pred,
    'Actual Price': y_price_test,
    'Predicted Price': y_price_pred
})

# 查看预测结果的前几行
print(results.head())

# 计算预测误差
results['Demand Error'] = results['Actual Demand'] - results['Predicted Demand']
results['Price Error'] = results['Actual Price'] - results['Predicted Price']

# 查看预测误差的统计信息
print(results.describe())

# 绘制预测结果与实际值的对比图
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))

# 电力需求预测结果对比图
plt.subplot(2, 1, 1)
plt.plot(results.index, results['Actual Demand'], label='Actual Demand')
plt.plot(results.index, results['Predicted Demand'], label='Predicted Demand')
plt.title('电力需求预测结果对比')
plt.xlabel('时间')
plt.ylabel('电力需求')
plt.legend()

# 电力价格预测结果对比图
plt.subplot(2, 1, 2)
plt.plot(results.index, results['Actual Price'], label='Actual Price')
plt.plot(results.index, results['Predicted Price'], label='Predicted Price')
plt.title('电力价格预测结果对比')
plt.xlabel('时间')
plt.ylabel('电力价格')
plt.legend()

plt.tight_layout()
plt.show()